### Esta todo en sucio y mucho sin explicar. Notebook de primera toma de contacto. El "modelo" está abajo del todo.

In [ ]:
pip install tornado --upgrade

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
os.environ['PATH'] = '/root/.local/bin:' + os.environ['PATH']


In [2]:
!pip install aicrowd-cli pyarrow --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!aicrowd login

Please login here: https://api.aicrowd.com/auth/uJveVXqgNuxn9RNTLM9tsDXxEc2kbRq-fHVU-CqPlVY
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://api.aicrowd.com/auth/uJveVXqgNuxn9RNTLM9tsDXxEc2kbRq-fHVU-CqPlVY'
API Key valid
Gitlab access token valid
Saved details successfully!


In [4]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [5]:
!aicrowd dataset download --challenge task-1-next-product-recommendation

sessions_test_task1.csv: 100% 19.4M/19.4M [00:02<00:00, 6.95MB/s]
sessions_test_task2.csv: 100% 1.92M/1.92M [00:01<00:00, 1.88MB/s]
sessions_test_task3.csv: 100% 2.67M/2.67M [00:01<00:00, 2.34MB/s]
products_train.csv: 100% 589M/589M [01:17<00:00, 7.57MB/s]
sessions_train.csv: 100% 259M/259M [00:48<00:00, 5.35MB/s]


In [6]:
import os
import numpy as np
import pandas as pd
from functools import lru_cache

In [7]:
train_data_dir = '.'
test_data_dir = '.'
task = 'task1'
PREDS_PER_SESSION = 100

In [31]:
# Cache loading of data for multiple calls

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(os.path.join(train_data_dir, 'products_train.csv'))

@lru_cache(maxsize=1)
def read_train_data():
    return pd.read_csv(os.path.join(train_data_dir, 'sessions_train.csv'))

@lru_cache(maxsize=3)
def read_test_data(task):
    return pd.read_csv(os.path.join(test_data_dir, f'sessions_test_{task}.csv'))

In [32]:
def read_locale_data(locale, task):
    products = read_product_data().query(f'locale == "{locale}"')
    sess_train = read_train_data().query(f'locale == "{locale}"')
    sess_test = read_test_data(task).query(f'locale == "{locale}"')
    return products, sess_train, sess_test

def show_locale_info(locale, task):
    products, sess_train, sess_test = read_locale_data(locale, task)

    train_l = sess_train['prev_items'].apply(lambda sess: len(sess))
    test_l = sess_test['prev_items'].apply(lambda sess: len(sess))

    print(f"Locale: {locale} \n"
          f"Number of products: {products['id'].nunique()} \n"
          f"Number of train sessions: {len(sess_train)} \n"
          f"Train session lengths - "
          f"Mean: {train_l.mean():.2f} | Median {train_l.median():.2f} | "
          f"Min: {train_l.min():.2f} | Max {train_l.max():.2f} \n"
          f"Number of test sessions: {len(sess_test)}"
        )
    if len(sess_test) > 0:
        print(
             f"Test session lengths - "
            f"Mean: {test_l.mean():.2f} | Median {test_l.median():.2f} | "
            f"Min: {test_l.min():.2f} | Max {test_l.max():.2f} \n"
        )
    print("======================================================================== \n")

In [ ]:
products = read_product_data()
locale_names = products['locale'].unique()
for locale in locale_names:
    show_locale_info(locale, task)

Locale: DE 
Number of products: 518327 
Number of train sessions: 1111416 
Train session lengths - Mean: 57.89 | Median 40.00 | Min: 27.00 | Max 2060.00 
Number of test sessions: 104568
Test session lengths - Mean: 57.23 | Median 40.00 | Min: 27.00 | Max 700.00 


Locale: JP 
Number of products: 395009 
Number of train sessions: 979119 
Train session lengths - Mean: 59.61 | Median 40.00 | Min: 27.00 | Max 6257.00 
Number of test sessions: 96467
Test session lengths - Mean: 59.90 | Median 40.00 | Min: 27.00 | Max 1479.00 


Locale: UK 
Number of products: 500180 
Number of train sessions: 1182181 
Train session lengths - Mean: 54.85 | Median 40.00 | Min: 27.00 | Max 2654.00 
Number of test sessions: 115936
Test session lengths - Mean: 53.51 | Median 40.00 | Min: 27.00 | Max 872.00 


Locale: ES 
Number of products: 42503 
Number of train sessions: 89047 
Train session lengths - Mean: 48.82 | Median 40.00 | Min: 27.00 | Max 792.00 
Number of test sessions: 0

Locale: FR 
Number of produc

In [33]:
train_sessions = read_train_data()
train_sessions.sample(5)

,prev_items,next_item,locale
3092800,['B0B5P3NZ8Y' 'B000ONIRIA'],B002KGLYLG,UK
3402121,['B09JR5M5LV' 'B0B3C66M8Z' 'B01M2AMPMU' 'B08D6...,B08FCSS581,FR
3455990,['B08F58P3G3' 'B08F58YYWY'],B097F9WDFJ,FR
3107684,['B07ZR187P5' 'B08F9M422Y' 'B0957GGSTP' 'B0957...,B07DGWSPKK,UK
1188093,['B09NBLXK8X' 'B094N7G4H6' 'B094N9XDLF' 'B09B8...,B09TVVN5VN,JP


In [34]:
test_sessions = read_test_data(task)
test_sessions.sample(5)

,prev_items,locale
135119,['B0058H6X7E' 'B0171AQ7TY' 'B0058H5J88'],JP
233476,['B08NVJZNLZ' 'B001RYNMDA' 'B0B3RMZ5YL' 'B001R...,UK
63513,['B097PLZZDT' 'B08CXP3WQB'],DE
178201,['B0B8RYJ59D' 'B0B8CZ7P3Z'],JP
302134,['B07VN49QF5' 'B089KKWW45' 'B00GQP0Z9E'],UK


In [35]:
df = test_sessions
df_productos=products
df_big = train_sessions

#Hago esto ya que había hecho todo el código usando esos nombres

In [ ]:
# NO EJECUTAR A NO SER QUE SE QUIERA HACER MRR CON UN CONJUNTO DE TEST PROPIO
from sklearn.model_selection import train_test_split
# Dividir el dataframe en conjuntos de entrenamiento y prueba
df_big, df = train_test_split(df_big, test_size=0.2, random_state=42)

# train_df es el conjunto de entrenamiento, que contiene el 80% de los datos originales
# test_df es el conjunto de prueba, que contiene el 20% de los datos originales


In [36]:
df['prev_items'] = df['prev_items'].str.replace('\n','').str.replace('[','').str.replace(']','').str.replace("'",'')


In [37]:
dominios_test = ['DE', 'JP', 'UK']

# Bucle for para crear los dataframes y agregarlos a la lista
for nombre in dominios_test:
    nombre_df = 'df_test_' + nombre  # Concatenar el nombre del dataframe con el sufijo correspondiente
    globals()[nombre_df] = df.loc[df['locale'] == nombre].copy()  # Crea el dataframe con el nombre correspondiente


# COSAS ***no ejecutar***

In [ ]:
df_big_alea = df_big.sample(n=500000, replace=True)

In [ ]:
#Escribimos las marcas de los items en una nueva columna.
marca_dict = dict(zip(df_productos['id'], df_productos['brand']))
def get_marcas(row):
    marcas = []
    for item in row['prev_items'].split():
        marca = marca_dict.get(item)
        if not pd.isna(marca):
            marcas.append(marca)
    return ' '.join(marcas)
df_big_alea['marcas'] = df_big_alea.apply(get_marcas, axis=1)


In [ ]:
df_big_alea['prev_items'] = df_big_alea['prev_items'].str.replace('\n','').str.replace('[','').str.replace(']','').str.replace("'",'')

dominios_train = df_big_alea['locale'].unique() 
# Bucle for para crear los dataframes y agregarlos a la lista
for nombre in dominios_train:
    nombre_df = 'df_train_' + nombre  # Concatenar el nombre del dataframe con el sufijo correspondiente
    nombre_productos='prods_' + nombre
    globals()[nombre_df] = df_big_alea.loc[df_big_alea['locale'] == nombre].copy()  # Crea el dataframe con el nombre correspondiente
    globals()[nombre_productos]= df_productos.loc[df_productos['locale'] == nombre].copy() 

In [ ]:
#Me centro solo en la base de datos de productos y las sesiones de UK.
display(df_train_UK.head())
display(prods_UK.head())

In [ ]:
# Separar los elementos de la columna prev_items por la coma y seleccionar el último elemento de cada lista
last_prev_item = df_train_UK['prev_items'].str.split(' ').str[-1]

# Crear el nuevo dataframe con la columna prev_item actualizada
new_df_train_UK = pd.DataFrame({'prev_item': last_prev_item, 'next_item': df_train_UK['next_item'], 'locale': df_train_UK['locale']})
new_df_train_UK

In [ ]:
# Contar las ocurrencias de cada last_item y seleccionar los 100 más frecuentes
last_item_counts = new_df_train_UK['prev_item'].value_counts().head(1000)

# Crear el nuevo dataframe con las columnas last_item, frequency y next_items
new_df = pd.DataFrame(columns=['last_item', 'frequency', 'next_items'])
new_df_train_UK
for last_item, frequency in last_item_counts.items():
    # Obtener la secuencia de next_items para cada last_item
    next_items = new_df_train_UK[new_df_train_UK['prev_item'] == last_item]['next_item'].values
    
    # Agregar la fila al nuevo dataframe
    new_df = new_df.append({'last_item': last_item, 'frequency': frequency, 'next_items': next_items}, ignore_index=True)



In [ ]:
from collections import Counter
porcentajes=[]

for i in range(len(new_df)):
    lista=list(new_df['next_items'][i])
    contador = Counter(lista)
    porcentajes.append(contador.most_common()[0][1]/new_df['frequency'][i])


In [ ]:
porcentajes
new_df

In [ ]:
# COGES EL ITEM QUE MÁS VECES SE VISITA DESPUÉS DE UN ITEM DADO Y GUARDAS SU PROBABILIDAD
# PEj SI VEO QUE INMEDIATAMENTE DESPUÉS DE VISITAR IPHONE14 EL USUARIO HA VISITADO FUNDA_IPHONE EN 10 DE LOS 20 CASOS EL PORCENTAJE 
# SERÁ 20
import matplotlib.pyplot as plt

# Crear la gráfica
plt.bar(range(len(porcentajes)), porcentajes)

# Personalizar la gráfica
plt.xticks(range(len(porcentajes)))
plt.xlabel('Elementos')
plt.ylabel('Valores')
plt.title('Gráfica de barras')

# Mostrar la gráfica
plt.show()

In [ ]:
[elemento for elemento in porcentajes if elemento > 0.8]

# CREAMOS MODELOS

In [38]:
df_big

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE
...,...,...,...
3606244,['B086CYFSKW' 'B0874F9859' 'B086CYFSKW'],B07B5TYD76,IT
3606245,['B09NRZKZ7V' 'B08WJTPV93'],B08L1P4C3D,IT
3606246,['B085JFX7MP' 'B085JGHW8R'],B01MPWVD44,IT
3606247,['B00B0UING2' 'B00B0UING2'],B00D3HYEZ4,IT


In [39]:
#Para el dataset de training
df_big=df_big[df_big['locale'].isin(['DE', 'UK', 'JP'])]
df_big['prev_items']=df_big['prev_items'].str.replace('\n','').str.replace('[','').str.replace(']','').str.replace("'",'')

# Dividir los prev_items por espacio en blanco y expandir en columnas
df_resumido_train=df_big.copy()
df_resumido_train['prev_items'] = df_resumido_train['prev_items'].str.split(' ')
df_resumido_train = df_resumido_train.explode('prev_items')

# Obtener el siguiente elemento de la lista prev_items
df_resumido_train['next_item'] = df_resumido_train.groupby('locale')['prev_items'].shift(-1)

# Eliminar filas con valores nulos en next_item
df_resumido_train.dropna(subset=['next_item'], inplace=True)

# Resetear el índice del dataframe
df_resumido_train.reset_index(drop=True, inplace=True)

# Seleccionar las columnas prev_items, next_item y locale
df_resumido_train = df_resumido_train[['prev_items', 'next_item', 'locale']]

# Renombrar las columnas
df_resumido_train.columns = ['prev_item', 'next_item', 'locale']

# Separar los elementos de la columna prev_items por la coma y seleccionar el último elemento de cada lista
last_prev_item = df_big['prev_items'].str.split(' ').str[-1]

# Crear el nuevo dataframe con la columna prev_item actualizada
df_big1 = pd.DataFrame({'prev_item': last_prev_item, 'next_item': df_big['next_item'], 'locale': df_resumido_train['locale']})

df_resumido_train=pd.concat([df_resumido_train,df_big1], ignore_index=False)

# Mostrar el nuevo dataframe
df_resumido_train

,prev_item,next_item,locale
0,B09W9FND7K,B09JSPLN1M,DE
1,B09JSPLN1M,B076THCGSG,DE
2,B076THCGSG,B007MO8IME,DE
3,B007MO8IME,B08MF65MLV,DE
4,B08MF65MLV,B001B4TKA0,DE
...,...,...,...
14098271,NaN,NaN,UK
14098272,NaN,NaN,UK
14098273,NaN,NaN,UK
14098274,NaN,NaN,UK


In [40]:
#Para el dataset de test
# Dividir los prev_items por espacio en blanco y expandir en columnas
df['prev_items']=df['prev_items'].str.replace('\n','').str.replace('[','').str.replace(']','').str.replace("'",'')
df_resumido_test=df
df_resumido_test['prev_items'] = df_resumido_test['prev_items'].str.split(' ')
df_resumido_test = df_resumido_test.explode('prev_items')

# Obtener el siguiente elemento de la lista prev_items
df_resumido_test['next_item'] = df_resumido_test.groupby('locale')['prev_items'].shift(-1)

# Eliminar filas con valores nulos en next_item
df_resumido_test.dropna(subset=['next_item'], inplace=True)

# Resetear el índice del dataframe
df_resumido_test.reset_index(drop=True, inplace=True)

# Seleccionar las columnas prev_items, next_item y locale
df_resumido_test = df_resumido_test[['prev_items', 'next_item', 'locale']]

# Renombrar las columnas
df_resumido_test.columns = ['prev_item', 'next_item', 'locale']


# Mostrar el nuevo dataframe
df_resumido_test

,prev_item,next_item,locale
0,B08V12CT4C,B08V1KXBQD,DE
1,B08V1KXBQD,B01BVG1XJS,DE
2,B01BVG1XJS,B09VC5PKN5,DE
3,B09VC5PKN5,B09V7KG931,DE
4,B09V7KG931,B09PY75FWM,DE
...,...,...,...
1351124,B09HL11V5B,B01MCQMORK,UK
1351125,B01MCQMORK,B09JYZ325W,UK
1351126,B09JYZ325W,B0B8JX92YJ,UK
1351127,B0B8JX92YJ,B09TN4MP6V,UK


In [41]:
df_big=pd.concat([df_resumido_train,df_resumido_test], ignore_index=False)
df_big

,prev_item,next_item,locale
0,B09W9FND7K,B09JSPLN1M,DE
1,B09JSPLN1M,B076THCGSG,DE
2,B076THCGSG,B007MO8IME,DE
3,B007MO8IME,B08MF65MLV,DE
4,B08MF65MLV,B001B4TKA0,DE
...,...,...,...
1351124,B09HL11V5B,B01MCQMORK,UK
1351125,B01MCQMORK,B09JYZ325W,UK
1351126,B09JYZ325W,B0B8JX92YJ,UK
1351127,B0B8JX92YJ,B09TN4MP6V,UK


In [42]:
# Crear un nuevo DataFrame para almacenar los datos procesados
df_procesado = df_big.groupby(['prev_item', 'next_item','locale']).size().reset_index(name='conteo')


# Ordenar el DataFrame por la columna "conteo" en orden descendente
df_procesado = df_procesado.sort_values(by='conteo', ascending=False)

# Mostrar el DataFrame ordenado
df_procesado[df_procesado["prev_item"]=="B0BFJGXWDV"] #por ejemplo

,prev_item,next_item,locale,conteo
9726279,B0BFJGXWDV,B0B1MPZWJG,DE,1110
9726294,B0BFJGXWDV,B0BFJGXWDV,DE,11
9726214,B0BFJGXWDV,B08H93ZRK9,DE,6
9726235,B0BFJGXWDV,B095Z1QGWJ,DE,4
9726299,B0BFJGXWDV,B0BG3HQZTQ,DE,4
...,...,...,...,...
9726269,B0BFJGXWDV,B09VXVQWH7,DE,1
9726268,B0BFJGXWDV,B09V83XQ12,DE,1
9726267,B0BFJGXWDV,B09V7KG931,DE,1
9726266,B0BFJGXWDV,B09TWH2PZZ,DE,1


In [43]:
for nombre in dominios_test:
    nombre_df = 'df_procesado_' + nombre  # Concatenar el nombre del dataframe con el sufijo correspondiente
    globals()[nombre_df] = df_procesado.loc[df_procesado['locale'] == nombre].copy()  # Crea el dataframe con el nombre correspondiente
    

In [44]:
df_procesado_UK

,prev_item,next_item,locale,conteo
5569068,B08PKLFM9T,B08F2NDB39,UK,872
825236,B008DJABEU,B08QDTSQY5,UK,636
4861379,B08CN3G4N9,B09BZR9JFG,UK,620
8569814,B0B18CK9DV,B0B18DBBFB,UK,562
4520664,B086LBQK8G,B01MDQQIKF,UK,549
...,...,...,...,...
3552423,B07S1897SM,B07B4J4RW7,UK,1
3552426,B07S1897SM,B09YXT5BQV,UK,1
3552428,B07S1897SM,B0B82YK9P8,UK,1
3552529,B07S18HLF7,B07S17DL3N,UK,1


In [45]:
def sistema_recomendacion(items_previos, data_locale):
    item_anterior = items_previos.split()[-1]
    siguientes_items = data_locale.get(item_anterior, [])
    return siguientes_items


In [ ]:
#ESTO SE PUEDE HACER MÁS EFICIENTE Y NO TAN FEO

# Crear un diccionario a partir de df_procesado_UK ya que trabajar con un diccionario es más eficiente que con csv
data_locale_dict_UK = df_procesado_UK.groupby("prev_item")["next_item"].apply(list).to_dict()
data_locale_dict_JP = df_procesado_JP.groupby("prev_item")["next_item"].apply(list).to_dict()
data_locale_dict_DE = df_procesado_DE.groupby("prev_item")["next_item"].apply(list).to_dict()

# Aplicar la función a un subconjunto de datos en df_test_UK
df_test_UK['next_item_prediction'] = df_test_UK["prev_items"].apply(sistema_recomendacion, data_locale=data_locale_dict_UK)
df_test_DE['next_item_prediction'] = df_test_DE["prev_items"].apply(sistema_recomendacion, data_locale=data_locale_dict_DE)
df_test_JP['next_item_prediction'] = df_test_JP["prev_items"].apply(sistema_recomendacion, data_locale=data_locale_dict_JP)


In [46]:
predictions = pd.concat([df_test_DE, df_test_JP, df_test_UK], ignore_index=False)

In [47]:
predictions

,prev_items,locale,next_item_prediction
0,B08V12CT4C B08V1KXBQD B01BVG1XJS B09VC5PKN5 B0...,DE,"[B099NS1XPG, B08496TCCQ, B01BVG1XJS, B099NR3X6..."
1,B00R9R5ND6 B00R9RZ9ZS B00R9RZ9ZS,DE,"[B004ZXMV4Q, B00R9R5ND6, B001LEO21W, B00R9RZ9Z..."
2,B07YSRXJD3 B07G7Q5N6G B08C9Q7QVK B07G7Q5N6G,DE,"[B08C9Q7QVK, B07G7Q5N6G, B07YSRXJD3, B0B5QNFWJ..."
3,B08KQBYV43 3955350843 3955350843 395535086X 39...,DE,"[395535086X, 3955350843, B0829LZFT1, B08KQBYV4..."
4,B09FPTCWMC B09FPTQP68 B08HMRY8NG B08TBBQ4BD B0...,DE,"[B09J8V18FL, B09J8T6TTH, B09J8SKX9G, B09J8TWRV..."
...,...,...,...
316966,B077SZ2C3Y B0B14M3VZX,UK,"[B08X9L5RGD, B08KFHDPY9, B09Y4HKGKT, B09MW64JG..."
316967,B08KFHDPY9 B0851KTSRZ B08KFHDPY9 B0851KTSRZ B0...,UK,"[B0989BHLSY, B09CPNS7XV, B09895QPQF, B09CPP92Q..."
316968,B07PY1N81F B07Q1Z8SQN B07PY1N81F B07Q1Z8SQN B0...,UK,"[B09HKZBNZH, B09HZSRJWW, B09HL141QC, B07PY1N81..."
316969,B01MCQMORK B09JYZ325W,UK,"[B09JYZ325W, B08FB464L7, B07TR5LQSL, B07KCQKHZ..."


# Código para testear MRR *no ejecutar si no hemos ejecutado la celda de dividir en train y test*

In [ ]:
mrr_list = []
for i, row in predictions.iterrows():
    relevant_item = row['next_item']
    prediction_list = row['next_item_prediction']
    try:
        rank = prediction_list.index(relevant_item) + 1  # Posición del ítem relevante en la lista de predicciones
        mrr = 1 / rank  # Calcular el MRR como el inverso de la posición del ítem relevante
    except ValueError:
        mrr = 0  # Si el ítem relevante no está en la lista de predicciones, asignar MRR 0
    mrr_list.append(mrr)

# Agregar la columna MRR al dataframe
predictions['MRR'] = mrr_list

In [ ]:
np.mean(predictions['MRR'])

KeyError: ignored

# CHECK Y ENVIAR

In [48]:
test_sessions=df
def check_predictions(predictions, check_products=False):
    """
    These tests need to pass as they will also be applied on the evaluator
    """
    test_locale_names = test_sessions['locale'].unique()
    for locale in test_locale_names:
        sess_test = test_sessions.query(f'locale == "{locale}"')
        preds_locale =  predictions[predictions['locale'] == sess_test['locale'].iloc[0]]
        assert sorted(preds_locale.index.values) == sorted(sess_test.index.values), f"Session ids of {locale} doesn't match"

        if check_products:
            # This check is not done on the evaluator
            # but you can run it to verify there is no mixing of products between locales
            # Since the ground truth next item will always belong to the same locale
            # Warning - This can be slow to run
            products = read_product_data().query(f'locale == "{locale}"')
            predicted_products = np.unique( np.array(list(preds_locale["next_item_prediction"].values)) )
            assert np.all( np.isin(predicted_products, products['id']) ), f"Invalid products in {locale} predictions"

In [49]:
check_predictions(predictions)

# Its important that the parquet file you submit is saved with pyarrow backend
predictions.to_parquet(f'submission_{task}.parquet', engine='pyarrow')


In [50]:
!aicrowd submission create -c task-1-next-product-recommendation -f "submission_task1.parquet"

Submission Upload Error: Something went wrong while uploading your submission
Submission limit reached for your account, it will reset at 2023-04-18 12:47:01 UTC
